# TensorFlow2 推理

下面以模型 [resnet_v2_50](http://download.tensorflow.org/models/resnet_v2_50_2017_04_14.tar.gz) 为例展示。

需要克隆项目 [models](https://github.com/tensorflow/models)，然后执行如下操作。

In [1]:
import tensorflow as tf
try:
    tf1 = tf.compat.v1
    tf1.disable_v2_behavior()
except (ImportError, AttributeError):
    tf1 = tf

2023-06-13 18:56:28.623811: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 18:56:28.672936: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 18:56:28.675980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 18:56:29.609843: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
non-resource variables are not supported in the long term


切换到 `models/research/slim` 目录下：

In [2]:
%cd /media/pc/data/lxw/ai/tasks/models/research/slim

/media/pc/data/lxw/ai/tasks/models/research/slim


定义缓存目录：

In [3]:
checkpoints_dir = '/tmp/checkpoints'

下载权重文件：

In [ ]:
from datasets import dataset_utils

url = "http://download.tensorflow.org/models/resnet_v2_50_2017_04_14.tar.gz"

if not tf1.gfile.Exists(checkpoints_dir):
    tf1.gfile.MakeDirs(checkpoints_dir)

dataset_utils.download_and_uncompress_tarball(url, checkpoints_dir)

## 升级 TF1 为 TF2

In [4]:
from PIL import Image
import numpy as np
from nets import resnet_v2
from preprocessing.preprocessing_factory import get_preprocessing
import tf_slim as slim


class ResnetV2_50(tf.keras.layers.Layer):
    def __init__(self, image_size, num_classes=1001, trainable=True, 
                 name="resnet_v2_50", dtype=None, dynamic=False, **kwargs):
        super().__init__(trainable, name, dtype, dynamic, **kwargs)
        self.image_size = image_size
        self.num_classes = num_classes
        self.preprocessing = get_preprocessing(self.name)

    @tf1.keras.utils.track_tf1_style_variables
    def call(self, inputs):
        processed_image = self.preprocessing(inputs, self.image_size, self.image_size)
        processed_images  = tf.expand_dims(processed_image/256, 0)
        
        # Create the model, use the default arg scope to configure the batch norm parameters.
        with slim.arg_scope(resnet_v2.resnet_arg_scope()):
            logits, _ = resnet_v2.resnet_v2_50(processed_images, 
                                               num_classes=self.num_classes,
                                               global_pool=True,
                                               is_training=self.trainable,
                                               scope=self.name)
        return tf.nn.softmax(logits)

In [9]:
model = ResnetV2_50(224)
x = tf.random.normal(shape=(224, 224, 3))
x = tf.cast(x, tf.uint8)
y = model(x)

Instructions for updating:
Colocations handled automatically by placer.


/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
from PIL import Image
import numpy as np
from nets import resnet_v2
from preprocessing.preprocessing_factory import get_preprocessing
import tf_slim as slim

preprocessing = get_preprocessing("resnet_v2_50")
image_size = 224
path = '/media/pc/data/board/arria10/lxw/data/test/cat.png' # 将要预测的图片路径

with Image.open(path) as im:
    if im.mode != "RGB":
        im.convert("RGB")
    image = np.asarray(im)


with tf1.Graph().as_default():
    image = tf.constant(image)
    processed_image = preprocessing(image, image_size, image_size)
    processed_images  = tf.expand_dims(processed_image/256, 0)
    
    # 创建模型时，使用默认的参数范围（arg scope）来配置批归一化（batch norm）参数。
    with slim.arg_scope(resnet_v2.resnet_arg_scope()):
        logits, _ = resnet_v2.resnet_v2_50(processed_images, num_classes=1001,
                                           global_pool=True,
                                           is_training=False)
    probabilities = tf.nn.softmax(logits)
    
    init_fn = slim.assign_from_checkpoint_fn(
        f'{checkpoints_dir}/resnet_v2_50.ckpt',
        slim.get_model_variables('resnet_v2_50'))
    
    with tf1.Session() as sess:
        init_fn(sess)
        np_processed_image, probabilities = sess.run([processed_image, probabilities])
        probabilities = probabilities[0, 0:]
        sorted_inds = [i[0] for i in sorted(enumerate(-probabilities), key=lambda x:x[1])]

打印标签信息：

In [ ]:
from github import Github


g = Github(user_agent="xinetzone")
repo = g.get_repo("tensorflow/models")
label_content = repo.get_contents("research/slim/datasets/imagenet_lsvrc_2015_synsets.txt")
imagenet_labels = label_content.decoded_content.decode().split()
assert len(imagenet_labels) == 1000
metadata = repo.get_contents("research/slim/datasets/imagenet_metadata.txt")
imagenet_metadata = metadata.decoded_content.decode().splitlines()
synset_to_human = {}
for metadata in imagenet_metadata:
    name, value = metadata.split("\t")
    synset_to_human[name] = value
name2id = {name: k+1 for k, name in enumerate(imagenet_labels)}

In [ ]:
topk = 5
for prob, sorted_ind in zip(probabilities[:topk], sorted_inds[:topk]):
    label = synset_to_human[imagenet_labels[sorted_ind-1]]
    print(f"{sorted_ind-1}: {label.ljust(20)}\t{prob}")